In [1]:
import socket
import requests
import json
import datetime
import time
import calendar

In [2]:
from configparser import ConfigParser
parser = ConfigParser()
_ = parser.read('notebook.cfg')
eventHubConnectString = parser.get('GetISPInfo_Notebook','eventHubConnectString')
eventhub_name ="ispinfo"

In [3]:
sleepMinutes =10

In [4]:

def GetTimestamp():
    gmTime= time.gmtime()    
    now = calendar.timegm( gmTime)
    print("GetTimestamp",now,gmTime)
    return now

In [5]:
def GetISPInfo():
    ipUrl = "http://ip-api.com/json"
    response= json.loads(requests.get(ipUrl).content)
    print(response)
    print(response["query"])
    return response

In [6]:
from azure.eventhub import EventHubProducerClient, EventData
def SendToEventHub(data):
    try:
        print("enter SendToEventHub", data)
        client = EventHubProducerClient.from_connection_string(eventHubConnectString, eventhub_name=eventhub_name)
        event_data_batch = client.create_batch()
        event_data_batch.add(EventData(data))
        with client:
            client.send_batch(event_data_batch)
        print("Sent data")
    except Exception as e:
        print("Unable to send data to event hub",e)

In [ ]:
import time
print("ready to start work")
linesToWrite = []
while (True):  
    try:
        ip = socket.gethostbyname(socket.gethostname())
        now = GetTimestamp()
        ispInfo= GetISPInfo()
        print(now,ispInfo)
    except  Exception as ex:
        print("exception",ex)
        now = GetTimestamp()
        ispInfo = None
        ip = socket.gethostbyname(socket.gethostname())
    eventData = json.dumps( {"utc":now,"ip":ip,"ispInfo":ispInfo})
    SendToEventHub(eventData)
    print("sent to event hub",eventData)
    if sleepMinutes>0:
        print('sleeping',sleepMinutes)
        time.sleep(sleepMinutes * 10) # time.sleep takes seconds as parameter

ready to start work
GetTimestamp 1689541187 time.struct_time(tm_year=2023, tm_mon=7, tm_mday=16, tm_hour=20, tm_min=59, tm_sec=47, tm_wday=6, tm_yday=197, tm_isdst=0)
{'status': 'success', 'country': 'United States', 'countryCode': 'US', 'region': 'NY', 'regionName': 'New York', 'city': 'New York', 'zip': '', 'lat': 40.7127, 'lon': -74.0059, 'timezone': 'America/New_York', 'isp': 'SpaceX Starlink', 'org': 'SpaceX Services, Inc', 'as': 'AS14593 Space Exploration Technologies Corporation', 'query': '129.222.216.91'}
129.222.216.91
1689541187 {'status': 'success', 'country': 'United States', 'countryCode': 'US', 'region': 'NY', 'regionName': 'New York', 'city': 'New York', 'zip': '', 'lat': 40.7127, 'lon': -74.0059, 'timezone': 'America/New_York', 'isp': 'SpaceX Starlink', 'org': 'SpaceX Services, Inc', 'as': 'AS14593 Space Exploration Technologies Corporation', 'query': '129.222.216.91'}
enter SendToEventHub {"utc": 1689541187, "ip": "172.18.160.1", "ispInfo": {"status": "success", "coun